## Where is the route info for January?
* Somehow the merge issue in notebook 47 was fixed (I didn't do anything??) 
* Now, all of the route names are gone for January.

In [1]:
import _section1_utils
import _section2_utils
import geopandas as gpd
import merge_data
import numpy as np
import pandas as pd
from segment_speed_utils import gtfs_schedule_wrangling
from shared_utils import rt_dates
from update_vars import GTFS_DATA_DICT, RT_SCHED_GCS, SCHED_GCS, SEGMENT_GCS

In [2]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [3]:
analysis_date_list = rt_dates.y2025_dates

In [4]:
df_sched = merge_data.concatenate_schedule_by_route_direction(analysis_date_list)

In [5]:
df_sched.head(1)

,schedule_gtfs_dataset_key,route_id,direction_id,time_period,route_primary_direction,avg_scheduled_service_minutes,avg_stop_miles,n_scheduled_trips,frequency,is_express,is_rapid,is_rail,is_coverage,is_downtown_local,is_local,service_date
0,0139b1253130b33adcd4b3a4490530d2,0177a66b-9f33-407d-a72e-776429fb73d4,0.00,all_day,Eastbound,63.50,4.38,2,0.08,0.00,0.00,0.00,1.00,0.00,0.00,2025-01-15


In [6]:
 df_avg_speeds = merge_data.concatenate_speeds_by_route_direction(
        analysis_date_list
    )

In [7]:
df_avg_speeds.head(1)

,schedule_gtfs_dataset_key,route_id,direction_id,time_period,speed_mph,service_date
0,015d67d5b75b5cf2b710bbadadfb75f5,17,0.00,all_day,13.46,2025-01-15


In [8]:
df_rt_sched = (
        merge_data.concatenate_rt_vs_schedule_by_route_direction(
            analysis_date_list
        ))

In [9]:
df_rt_sched.head(2)

,schedule_gtfs_dataset_key,route_id,direction_id,time_period,minutes_atleast1_vp,minutes_atleast2_vp,total_rt_service_minutes,total_scheduled_service_minutes,total_vp,vp_in_shape,is_early,is_ontime,is_late,n_vp_trips,vp_per_minute,pct_in_shape,pct_rt_journey_atleast1_vp,pct_rt_journey_atleast2_vp,pct_sched_journey_atleast1_vp,pct_sched_journey_atleast2_vp,rt_sched_journey_ratio,avg_rt_service_minutes,name,schedule_source_record_id,service_date
0,015d67d5b75b5cf2b710bbadadfb75f5,17,0.00,all_day,1403,1376,1567.85,1045.00,4127,3138,1,1,18,20,2.63,0.76,0.90,0.88,1.00,1.00,1.50,78.39,Bay Area 511 Marin Schedule,reckCEnFkdLVgfxck,2025-01-15
1,015d67d5b75b5cf2b710bbadadfb75f5,17,0.00,offpeak,806,789,956.95,569.00,2371,1797,0,0,11,11,2.48,0.76,0.84,0.82,1.00,1.00,1.68,87.00,Bay Area 511 Marin Schedule,reckCEnFkdLVgfxck,2025-01-15


In [10]:
df_crosswalk = merge_data.concatenate_crosswalk_organization(
        analysis_date_list
)

In [11]:
df_crosswalk.head(1)

,schedule_gtfs_dataset_key,name,schedule_source_record_id,base64_url,organization_source_record_id,organization_name,caltrans_district,service_date
0,ff1bc5dde661d62c877165421e9ca257,Santa Ynez Mecatran Schedule,recuWhPXfxMatv6rL,aHR0cDovL2FwcC5tZWNhdHJhbi5jb20vdXJiL3dzL2ZlZWQvYzJsMFpUMXplWFowTzJOc2FXVnVkRDF6Wld4bU8yVjRjR2x5WlQwN2RIbHdaVDFuZEdaek8ydGxlVDAwTWpjd056UTBaVFk0TlRBek9UTXlNREl4TURkak56STBNRFJrTXpZeU5UTTRNekkwWXpJMA==,reckp33bhAuZlmO1M,City of Solvang,05 - San Luis Obispo,2025-01-15


In [12]:
primary_typology = merge_data.set_primary_typology(df_sched)

In [13]:
primary_typology.head(1)

,schedule_gtfs_dataset_key,route_id,direction_id,time_period,typology
0,0139b1253130b33adcd4b3a4490530d2,0177a66b-9f33-407d-a72e-776429fb73d4,0.00,all_day,coverage


In [14]:
df_schedule2 = pd.merge(
        df_sched,
        primary_typology,
        on = merge_data.route_time_cols,
        how = "left"
    )

In [15]:
df_schedule2.head(2)

,schedule_gtfs_dataset_key,route_id,direction_id,time_period,route_primary_direction,avg_scheduled_service_minutes,avg_stop_miles,n_scheduled_trips,frequency,is_express,is_rapid,is_rail,is_coverage,is_downtown_local,is_local,service_date,typology
0,0139b1253130b33adcd4b3a4490530d2,0177a66b-9f33-407d-a72e-776429fb73d4,0.00,all_day,Eastbound,63.50,4.38,2,0.08,0.00,0.00,0.00,1.00,0.00,0.00,2025-01-15,coverage
1,0139b1253130b33adcd4b3a4490530d2,0177a66b-9f33-407d-a72e-776429fb73d4,0.00,offpeak,Eastbound,63.50,4.38,1,0.04,0.00,0.00,0.00,1.00,0.00,0.00,2025-01-15,coverage


In [16]:
df = pd.merge(
        df_schedule2,
        df_rt_sched,
        on = merge_data.route_time_cols + ["service_date"],
        how = "outer",
        indicator = "sched_rt_category"
    ).merge(
        df_avg_speeds,
        on = merge_data.route_time_cols + ["service_date"],
        how = "outer",
    )

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14996 entries, 0 to 14995
Data columns (total 39 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   schedule_gtfs_dataset_key        14996 non-null  object        
 1   route_id                         14996 non-null  object        
 2   direction_id                     14996 non-null  float64       
 3   time_period                      14996 non-null  object        
 4   route_primary_direction          14060 non-null  object        
 5   avg_scheduled_service_minutes    14060 non-null  float64       
 6   avg_stop_miles                   13965 non-null  float64       
 7   n_scheduled_trips                14060 non-null  float64       
 8   frequency                        14060 non-null  float64       
 9   is_express                       13379 non-null  float64       
 10  is_rapid                         13379 non-null  float64  

In [18]:
df.sample(3)

,schedule_gtfs_dataset_key,route_id,direction_id,time_period,route_primary_direction,avg_scheduled_service_minutes,avg_stop_miles,n_scheduled_trips,frequency,is_express,is_rapid,is_rail,is_coverage,is_downtown_local,is_local,service_date,typology,minutes_atleast1_vp,minutes_atleast2_vp,total_rt_service_minutes,total_scheduled_service_minutes,total_vp,vp_in_shape,is_early,is_ontime,is_late,n_vp_trips,vp_per_minute,pct_in_shape,pct_rt_journey_atleast1_vp,pct_rt_journey_atleast2_vp,pct_sched_journey_atleast1_vp,pct_sched_journey_atleast2_vp,rt_sched_journey_ratio,avg_rt_service_minutes,name,schedule_source_record_id,sched_rt_category,speed_mph
14031,fe4aab1717eca5a2935c32c85a35a5bf,5,0.00,all_day,Northbound,43.33,16.81,6.00,0.25,0.00,0.00,0.00,1.00,0.00,0.00,2025-01-15,coverage,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,NaN
13398,f948cebbdadd3824ca220cf696364392,1C,0.00,peak,Eastbound,24.00,2.75,2.00,0.08,0.00,1.00,0.00,0.00,1.00,0.00,2025-01-15,rapid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,NaN
12594,efbbd5293be71f7a5de0cf82b59febe1,3807,0.00,all_day,Westbound,57.92,8.31,78.00,3.25,0.00,0.00,0.00,0.00,1.00,0.00,2025-01-15,downtown_local,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,NaN


In [19]:
df2 = df.assign(
        sched_rt_category = df.sched_rt_category.map(
            gtfs_schedule_wrangling.sched_rt_category_dict)
    )

In [20]:
df2.sched_rt_category.value_counts()

schedule_and_vp    11205
schedule_only       2855
vp_only              936
Name: sched_rt_category, dtype: int64

### `merge_in_standardized_route_names` isn't working -> break it out.

In [21]:
df3 = df2.pipe(
        merge_data.merge_in_standardized_route_names,
    )

In [22]:
df3.sample(3)

,schedule_gtfs_dataset_key,direction_id,time_period,route_primary_direction,avg_scheduled_service_minutes,avg_stop_miles,n_scheduled_trips,frequency,is_express,is_rapid,is_rail,is_coverage,is_downtown_local,is_local,service_date,typology,minutes_atleast1_vp,minutes_atleast2_vp,total_rt_service_minutes,total_scheduled_service_minutes,total_vp,vp_in_shape,is_early,is_ontime,is_late,n_vp_trips,vp_per_minute,pct_in_shape,pct_rt_journey_atleast1_vp,pct_rt_journey_atleast2_vp,pct_sched_journey_atleast1_vp,pct_sched_journey_atleast2_vp,rt_sched_journey_ratio,avg_rt_service_minutes,schedule_source_record_id,sched_rt_category,speed_mph,name,route_long_name,route_short_name,route_combined_name,route_id
8638,d2b09fbd392b28d767c28ea26529b0cd,0.00,peak,Westbound,7.00,8.31,15.00,0.62,1.00,0.00,0.00,1.00,0.00,0.00,2025-01-15,express,12.00,11.00,11.18,7.00,28.00,28.00,0.00,1.00,0.00,1.00,2.50,1.00,1.00,0.98,1.00,1.00,1.60,11.18,recpwl8uJU04SPbWS,schedule_and_vp,NaN,Unitrans Schedule,West Village Limited - Sol & Ramble Only,VL,VL West Village Limited - Sol & Ramble Only,VL
8133,c499f905e33929a641f083dad55c521e,0.00,all_day,Eastbound,38.56,3.54,41.00,1.71,0.00,1.00,0.00,0.00,1.00,0.00,2025-01-15,rapid,2325.00,2279.00,2287.18,1581.00,6586.00,6401.00,0.00,1.00,40.00,41.00,2.88,0.97,1.00,1.00,1.00,1.00,1.45,55.78,recJjD8JT53sK302o,schedule_and_vp,14.83,Bay Area 511 AC Transit Schedule,Winton - Industrial - Tennyson,86,86 Winton - Industrial - Tennyson,86
6994,baeeb157e85a901e47b828ef9fe75091,0.00,all_day,Eastbound,34.02,5.54,50.00,2.08,0.00,0.00,0.00,0.00,1.00,0.00,2025-01-15,downtown_local,1998.00,1739.00,1982.90,1701.00,4306.00,4110.00,0.00,21.00,29.00,50.00,2.17,0.95,1.00,0.88,1.00,1.00,1.17,39.66,recfZ9iWkptccoONX,schedule_and_vp,13.76,San Diego Schedule,Palomar Transit Center - Southwestern College,712,712 Palomar Transit Center - Southwestern College,712


In [23]:
keep_cols = [
        "schedule_gtfs_dataset_key", "name", 
        "route_id", "service_date", 
    ]
    

In [24]:
CLEAN_ROUTES = GTFS_DATA_DICT.schedule_tables.route_identification

In [25]:
route_names_df = pd.read_parquet(
        f"{SCHED_GCS}{CLEAN_ROUTES}.parquet"
    )

In [26]:
route_names_df.service_date.unique()

array(['2024-01-17T00:00:00.000000000', '2024-02-14T00:00:00.000000000',
       '2024-03-13T00:00:00.000000000', '2024-04-17T00:00:00.000000000',
       '2024-05-22T00:00:00.000000000', '2024-06-12T00:00:00.000000000',
       '2024-07-17T00:00:00.000000000', '2024-08-14T00:00:00.000000000',
       '2024-09-18T00:00:00.000000000', '2024-10-16T00:00:00.000000000',
       '2024-11-13T00:00:00.000000000', '2024-12-11T00:00:00.000000000',
       '2023-03-15T00:00:00.000000000', '2023-04-12T00:00:00.000000000',
       '2023-05-17T00:00:00.000000000', '2023-06-14T00:00:00.000000000',
       '2023-07-12T00:00:00.000000000', '2023-08-15T00:00:00.000000000',
       '2023-09-13T00:00:00.000000000', '2023-10-11T00:00:00.000000000',
       '2023-11-15T00:00:00.000000000', '2023-12-13T00:00:00.000000000',
       '2023-10-09T00:00:00.000000000', '2023-10-10T00:00:00.000000000',
       '2023-10-12T00:00:00.000000000', '2023-10-13T00:00:00.000000000',
       '2023-10-14T00:00:00.000000000', '2023-10-15

In [27]:
route_names_df.head(2)

,schedule_gtfs_dataset_key,name,route_id,route_long_name,route_short_name,route_desc,service_date,combined_name,route_id2,recent_combined_name,recent_route_id2
0,1770249a5a2e770ca90628434d4934b1,VCTC GMV Schedule,3407,Route 18,Route 18,OHS TRIPPER - C ST - DORIS ST,2024-01-17,Route 18__Route 18,3407,Route 18__Route 18,3407
1,1770249a5a2e770ca90628434d4934b1,VCTC GMV Schedule,3408,Route 21,Route 21,PACIFIC VIEW MALL via VICTORIA AVE,2024-01-17,Route 21__Route 21,3408,Route 21__Route 21,3408
